### Predictive Maintenance in the Robotic Arms Industry by applying Digital Twin Technology

This is the main working file. It includes a function that simulates a robotic arm. During its operation, there will be irregularities. The applied algorithms will try to detect the irregularities, classify them as anomalies, and set further measures to pinpoint the problematic component.

## Imports

Before running the notebook, make sure that all neccessary modules and programms are correctly installed. Also, I would advise to use Python 3.7.9, since I had troubles running parts of the code with other versions.

In [1]:
#machine learning
import shap
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import pyspark

#azure
from azure.digitaltwins.core import DigitalTwinsClient
from azure.identity import DefaultAzureCredential
from azure.identity import VisualStudioCodeCredential

#additional installs
import os
import time

#python scripts
import anomaly_detection as ad
import digital_twin_azure as dt
import predictive_maintenance as pm

## Connecting to Azure 

The following code connects your environment to the Azure platform. A browser window will pop up and you will be asked to login to your microsoft account. This information will then be stored for easy access of to the SDK.

In [2]:
!az login

[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "0504f721-d451-402b-b884-381428559e39",
    "id": "2320a8eb-1fce-45ea-9dec-ce93cf71ea5c",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Azure for Students",
    "state": "Enabled",
    "tenantId": "0504f721-d451-402b-b884-381428559e39",
    "user": {
      "name": "h1548782@s.wu.ac.at",
      "type": "user"
    }
  }
]


## Credentials

The following code chunk uses the credentials received by the previous step. It will build a service client which will be needed to update the Digital Twin, or run certain queries.

In [3]:
#define the URL of your Digital Twin instance on the Azure platzform
url = "SeleniumForest.api.weu.digitaltwins.azure.net"

#store the gathered credentials in a variable
credential = DefaultAzureCredential()
#create an instance of the Digital Twin Client
#It can be resued later on
global service_client
service_client = DigitalTwinsClient(url, credential)

## Load Data Set

Next we will load the data set into a pandas datafram. Make sure that the Data repository exists within the set working directory. 

In [2]:
#store the CSV content in a dataframe
df = pd.read_csv('./Data/right_arm.csv')
df.head()

,Timestamp,Actual Joint Positions,Actual Joint Velocities,Actual Joint Currents,Actual Cartesian Coordinates,Actual Tool Speed,Generalized Forces,Temperature of Each Joint,Execution Time,Safety Status,Tool Acceleration,Norm of Cartesion Linear Momentum,Robot Current,Joint Voltages,Elbow Position,Elbow Velocity,Tool Current,Tool Temperature,TCP Force,Anomaly State
0,257258.126,"[-1.5707390944110315, -1.5707948964885254, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.18499481678009033, 1.1043120622634888, 1.38...","[-0.133089990415502, -0.4433227297722474, 0.48...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[-0.015098106867198664, -0.38619858377727334, ...","[34.375, 36.875, 39.6875, 42.8125, 45.0, 45.3125]",0.833027,1,"[-9.423578262329102, 0.26815059781074524, 0.11...",0.000000,0.560617,"[47.48460006713867, 47.61359786987305, 47.6279...","[-0.07999999993886807, -4.2302395934749395e-06...","[0.0, 0.0, 0.0]",0.069408,39.6250,0.693609,0
1,257258.176,"[-1.5707176367389124, -1.570843359033102, -1.5...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.16534548997879028, 1.0664139986038208, 1.40...","[-0.13308335724829182, -0.44333549960206997, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[-0.04213328588448917, -0.4293152706290817, 0....","[34.375, 36.875, 39.6875, 42.8125, 45.0, 45.3125]",0.898871,1,"[-9.346963882446289, 0.2298433631658554, 0.114...",0.000000,0.497956,"[47.5275993347168, 47.64226150512695, 47.65659...","[-0.07999999890089683, -1.7749906051421094e-05...","[0.0, 0.0, 0.0]",0.064909,39.6250,0.685211,0
2,257258.226,"[-1.570730988179342, -1.5708185635008753, -1.5...","[0.0, 0.0, 0.0, -0.010884314775466919, 0.0, 0.0]","[0.1645040214061737, 1.0770467519760132, 1.354...","[-0.13307894938257267, -0.44333179729023336, 0...","[-2.003684872597315e-06, -0.000926684519370717...","[0.05686369196465915, -0.2101876951251029, 0.9...","[34.375, 36.875, 39.6875, 42.8125, 45.0, 45.3125]",1.048140,1,"[-9.385271072387695, 0.2298433631658554, 0.191...",0.004019,0.504669,"[47.513267517089844, 47.670928955078125, 47.67...","[-0.07999999952532996, -1.0642838973058337e-05...","[0.0, 0.0, 0.0]",0.071274,39.6250,0.944428,0
3,257258.276,"[-1.570660416279928, -1.5707827371409913, -1.5...","[0.0, 0.0, 0.0, -0.08753828704357147, 0.0, 0.0]","[0.18218308687210083, 1.0735350847244263, 1.37...","[-0.13305178855972016, -0.443525353390802, 0.4...","[-1.5362262575268616e-05, -0.00740704821459850...","[0.014812691800918066, -0.0035357919273737524,...","[34.375, 36.875, 39.6875, 42.8125, 45.0, 45.3125]",1.124447,1,"[-9.327810287475586, 0.2298433631658554, 0.191...",0.026350,0.506904,"[47.55626678466797, 47.713924407958984, 47.670...","[-0.07999999976091744, -7.5630809928628735e-06...","[0.0, 0.0, 0.0]",0.071954,39.6250,0.891657,0
4,257258.326,"[-1.570700470601217, -1.570839067498678, -1.57...","[0.0, 0.0, 0.0, -0.14845426380634308, 0.0, 0.0]","[0.1679745316505432, 1.0931836366653442, 1.374...","[-0.13306762017073243, -0.44405501412764015, 0...","[-2.5524599112393154e-05, -0.01234826561117640...","[-0.0066400665555922875, -0.33964563763481687,...","[34.375, 36.875, 39.6875, 42.75, 45.0, 45.3125]",1.072475,1,"[-9.404424667358398, 0.3064578175544739, 0.057...",0.048680,0.457674,"[47.55626678466797, 47.670928955078125, 47.685...","[-0.07999999868460166, -1.8077993842428327e-05...","[0.0, 0.0, 0.0]",0.066182,39.6875,0.750271,0


## Pre-Processing

This section will cover the pre-processing of the available dataframe. First, all blank spaces within the column names will be replaced by underlines. This will make working with the names a lot easier, as it allows for simple copy and paste shortcuts and avoid processing errors. Furthermore, the function extract_every_nth_row() can be used to reduce the proccessing time. It extracts every n´th row of the dataframe. Since one might run low computational resources, n can be adjusted to personal preferences or skipped entirely. 

Furthermore, there will  be a train and test split of all defined features and the target. Lastly, the index will be reseted.

In [3]:
#replace blank spaces in the column names with '_'
df.columns = df.columns.str.replace(' ', '_')

#use only required columns
df = df[['Norm_of_Cartesion_Linear_Momentum', 'Robot_Current', 'Tool_Current', 'Tool_Temperature', 'TCP_Force', 'Anomaly_State']]

################################
##make it smaller for testing###
#since the dataset is quite big 
#one can keep this code block, to
#shrink the dataset#############
def extract_every_nth_row(df, n):
    new_df = df.iloc[::n].copy()
    return new_df

n = 100

df = extract_every_nth_row(df, n)
################################
################################

#seperate the features and the target
#X and y 
#features
X = df[['Norm_of_Cartesion_Linear_Momentum', 'Robot_Current', 'Tool_Current', 'Tool_Temperature', 'TCP_Force']]
#target
y = df['Anomaly_State'] 

#Train & Test Split
#for now the testsize is 20% of the total dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#reset the index to make the subsets iterateable again
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

## Simulation

The following code will focus on creating and running the simulation. The function start_machine() contains all previously created python scripts. By running start_machine() the function will simulate the startup of the robotic arm. Sensory data coming from teh simulation data set will be received. 

First, the code will make sure that the environment is correctly connected to the Azure instance. A browser window might pop-up and you will be asked to login with your personal credentials. Next, train_model() will be used to build a random forest prediction model. After that step is completed, simulated robotic arm will start sending sensory data to the digital environment. By running predict_model(), the algorithm will predict the current anomaly state for the machine given the pre-trained model. After that, the received prediction will be sent to the Digital Twin together with all received sensory data. Additionally, a dashboard-like plot will be displayed, showing all current workloads and the machines anomaly state. In case an anomaly is detected, an alarm will pop-up notifing the user. Furthermore, a SHAP bar plot will appear. This plot will show which component was relevant for the algorithm´s predictions. Domain-expert therefore, can take a closer look at the identified faulty components. 

This can be done untill the simulation data runs out, or the process has reached the predefined threshold (see "set iterations for simulation run").  

In [16]:
def start_machine(model_name: str, df, df_sim, index):
        print('Connecting to the Azure platform...')
        service_client = dt.connect_azure()
        
        print('Machine starting up...')
        print('Training ML algorithm...')
        rf = ad.train_model(X_train, X_test, y_train, y_test)
        
        print('Model has been trained')
        
        #set as global to calculate resulting scores
        global y_predicted
        
        #set empty list to store predictions
        y_predicted_list = []

        #set iterations for simulation run
        while index < 5:
                y_predicted = ad.predict_model(X_test, index, rf)
                y_predicted_list.append(y_predicted)
                dt.update_machine('RoboArm', X_test, index, y_predicted)
                dt.plot_twin_state()
                
                if y_predicted == 1:
                        print('Anomaly detected!')
                else:
                        print('No anomalie detected.')
                
                index += 1
        
        print('Simulation complete. Generate SHAP beeswarm and partial dependence plot...')
        
        #Generate beeswarm plot
        #beeswarm_plot = pm.create_shap_beeswarm_all(rf, X_test, y_predicted_list)
        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)
        y_predicted = model.predict(X_test)

        explainer = shap.Explainer(model)
        shap_values = explainer(X_test)

        positive_indices = np.where(y_predicted == 1)[0]
        shap_values_positive = shap_values[positive_indices, :, 1]

        beeswarm = shap.plots.beeswarm(shap_values_positive)
        
        #Generate partial dependence plot
        #This code generates a SHAP partial dependence plot for a specific instance.
        #The instance can be chosen by chaning the sample_ind value.
        sample_ind = 0
        pdp = shap.partial_dependence_plot(
           "Tool_Temperature", model.predict, X_test, model_expected_value=True,
            feature_expected_value=True, ice=False,
            shap_values=shap_values_positive[sample_ind:sample_ind+1,:]
        )
                
        return beeswarm, pdp

The following code chunk will start the machine. By defining the initial index the starting point within the simulation data set can be chosen. In this case, the X_test set was used as the simulation dataset, giving the possibility to compare prediction results with actual anomalies.

It is worth mentioning, that SHAP might have issues displaying the plot due to interdepenencies to other plot modules. For most cases, the shap.initjs() should fix this issue. However, if the issue still occurs I would recommend storing the plot as a jpg or png and open it manually on your machine instead of within the IDE.

In [17]:
#set starting index for simulation data set
index = 0
df = df
X_test
shap.initjs()

start_machine('RoboArm', df, X_test, index)

No anomalie detected.
Simulation complete. Generate SHAP beeswarm and partial dependence plot...


(None, None)

### Simulation Results
The following code can be used to make working and evaluating the digital twin´s performance a bit easier. 

In [ ]:
###SIMULATION NO VISUALIZATIONS to calculate simulation results
#this runs the previous defined function
#plots are commented out
#also stores all generated predictions for performance metrics calculation
#be aware that running through the simulation will take about 10min
def start_machine(model_name: str, df, df_sim, index):
    print('Connecting to the Azure platform...')
    service_client = dt.connect_azure()

    print('Machine starting up...')
    print('Training ML Algorithm...')
    rf = ad.train_model(X_train, X_test, y_train, y_test)

    print('Model has been trained')

    # Create an empty list to store the prediction results
    global prediction_results
    prediction_results = []

    # set iterations for simulation run
    while index < 200:
        y_predicted = ad.predict_model(X_test, index, rf)
        dt.update_machine('RoboArm', X_test, index, y_predicted)
        #dt.plot_twin_state()

        # Append the predicted value to the list
        prediction_results.append(y_predicted)

        if y_predicted == 1:
            print('Anomaly detected! Print SHAP plot...')
            #pm.explain_prediction(X_test, index, rf)
        else:
            print('No anomaly detected.')

        index += 1
        
    #pm.shap_test(X_test, index, rf) #####delete tghis afterwards    
    

    return prediction_results

In [ ]:
cm = confusion_matrix(y_test, prediction_results)
print("Confusion Matrix:")
print(cm)

#Accuracy of Simulation
accuracy = accuracy_score(y_test, prediction_results)
print("Accuracy:", accuracy)

#Precision of Simulation
precision = precision_score (y_test, prediction_results)
print("Precision:", precision)


comparison_df = pd.DataFrame({
        'y_test': y_test, 
        'prediction_results': prediction_results
    })
comparison_df.head(50)

NameError: name 'prediction_results' is not defined

### Beeswarm Plot to visualize SHAP values during Simulation

In [4]:
#This code will print a SHAP beeswarm plot of the simulation data (X_test)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)

explainer = shap.Explainer(model)
shap_values = explainer(X_test)

positive_indices = np.where(y_predicted == 1)[0]
shap_values_positive = shap_values[positive_indices, :, 1]

shap.plots.beeswarm(shap_values_positive, plot_size=(10, 6))

### Partial Dependence Plot 

In [11]:
#This code generates a SHAP partial dependence plot for a specific instance.
#The instance can be chosen by chaning the sample_ind value.
sample_ind = 0
shap.partial_dependence_plot(
    "Tool_Temperature", model.predict, X_test, model_expected_value=True,
    feature_expected_value=True, ice=False,
    shap_values=shap_values_positive[sample_ind:sample_ind+1,:]
)

In [12]:
X_test.shape

(3526, 5)